In [26]:
from pathlib import Path
from kilosort.io import load_ops
import sys
import spikeinterface as si
import matplotlib.pyplot as plt

import spikeinterface.extractors as se
import spikeinterface.preprocessing as spre
import spikeinterface.sorters as ss
import spikeinterface.widgets as sw
import spikeinterface.qualitymetrics as sqm
import json
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
from kilosort import io
warnings.filterwarnings('ignore')

global_job_kwargs = dict(n_jobs = 4)
si.set_global_job_kwargs(**global_job_kwargs)

In [27]:
import os
import pandas as pd
import numpy as np

def get_spike_inf(file_path):
    cluster_inf = pd.read_csv(file_path + "/analyzer_kilosort4_binary/extensions/quality_metrics/metrics.csv")
    cluster_inf.columns = ['cluster', 'num_spikes', 'firing_rate', 'presence_ratio', 'snr',
                           'isi_violations_ratio', 'isi_violations_count', 'rp_contamination',
                           'rp_violations', 'sliding_rp_violation', 'amplitude_cutoff',
                           'amplitude_median', 'amplitude_cv_median', 'amplitude_cv_range',
                           'sync_spike_2', 'sync_spike_4', 'sync_spike_8', 'firing_range',
                           'drift_ptp', 'drift_std', 'drift_mad', 'sd_ratio']
    
    cluster_inf['cluster'] = cluster_inf['cluster'].astype(str)
    cluster_inf['position_1'] = None
    cluster_inf['position_2'] = None

    spike_clusters = pd.DataFrame(np.load(file_path + "/kilosort4/sorter_output/spike_clusters.npy").astype(str))
    spike_positions = pd.DataFrame(np.load(file_path + "/kilosort4/sorter_output/spike_positions.npy").astype(float))
    spike_templates = pd.DataFrame(np.load(file_path + "/kilosort4/sorter_output/spike_templates.npy"))
    spike_times = pd.DataFrame(np.load(file_path + "/kilosort4/sorter_output/spike_times.npy").astype(int))
    tf = pd.DataFrame(np.load(file_path + "/kilosort4/sorter_output/tF.npy")[:, 0, :])

    spike_inf = pd.concat((spike_clusters, spike_positions, spike_templates, spike_times, tf), axis=1)
    spike_inf.columns = ['cluster', 'position_1', 'position_2', 'templates', 'time', 'PC_1', 'PC_2', 'PC_3', 'PC_4', 'PC_5', 'PC_6']

    for i in spike_inf['cluster'].value_counts().index:
        temp = spike_inf[spike_inf['cluster'] == i]
        cluster_inf.loc[cluster_inf['cluster'] == i, 'position_1'] = np.mean(temp['position_1'])
        cluster_inf.loc[cluster_inf['cluster'] == i, 'position_2'] = np.mean(temp['position_2'])

    cluster_inf['probe_group'] = "1"

    for i in spike_inf['cluster'].value_counts().index:
        cluster_rows = cluster_inf[cluster_inf['cluster'] == i]
        if (cluster_rows['position_1'] > 100).any() and (cluster_rows['position_1'] < 250).any():
            cluster_inf.loc[cluster_inf['cluster'] == i, 'probe_group'] = "2"
        elif (cluster_rows['position_1'] > 250).any() and (cluster_rows['position_1'] < 400).any():
            cluster_inf.loc[cluster_inf['cluster'] == i, 'probe_group'] = "3"
        elif (cluster_rows['position_1'] > 400).any() and (cluster_rows['position_1'] < 550).any():
            cluster_inf.loc[cluster_inf['cluster'] == i, 'probe_group'] = "4"
        elif (cluster_rows['position_1'] > 550).any():
            cluster_inf.loc[cluster_inf['cluster'] == i, 'probe_group'] = "5"

    waveform = np.load(file_path + "/kilosort4/sorter_output/templates.npy")
    cluster_inf['waveform'] = [waveform[i] for i in range(waveform.shape[0])]

    cluster_inf = cluster_inf[((cluster_inf['snr'] > 3) & (cluster_inf['num_spikes'] > int(5000))) | ((cluster_inf['snr'] < 3) & (cluster_inf['num_spikes'] > 8000))]
    spike_inf = spike_inf[spike_inf['cluster'].isin(list(cluster_inf['cluster']))]
    spike_inf = spike_inf[spike_inf['time'] > 200]
    cluster_inf['date'] = date
    spike_inf['date'] = date
    
    channel_indices = {
        "1": [1, 3, 5, 6, 9, 11],
        "2": [13, 15, 17, 19, 21, 23],
        "3": [24, 25, 26, 27, 28, 29],
        "4": [12, 14, 16, 18, 20, 22],
        "5": [0, 2, 4, 6, 8, 10, 12]
        }

    for index, row in cluster_inf.iterrows():
        probe_group = row['probe_group']
        if probe_group in channel_indices:
            selected_channels = channel_indices[probe_group]
            cluster_inf.at[index, 'waveform'] = row['waveform'][:, selected_channels]

    return cluster_inf, spike_inf

all_cluster_inf = pd.DataFrame()
all_spike_inf = pd.DataFrame()

for date in os.listdir("/media/ubuntu/sda/data/sort_output/mouse21/"):
    cluster_inf, spike_inf = get_spike_inf(file_path=f"/media/ubuntu/sda/data/sort_output/mouse21/{date}")
    all_cluster_inf = pd.concat([all_cluster_inf, cluster_inf], ignore_index=True)
    all_spike_inf = pd.concat([all_spike_inf, spike_inf], ignore_index=True)

In [28]:
channel_indices = {
        "1": [1, 3, 5, 6, 9, 11],
        "2": [13, 15, 17, 19, 21, 23],
        "3": [24, 25, 26, 27, 28, 29],
        "4": [12, 14, 16, 18, 20, 22],
        "5": [0, 2, 4, 6, 8, 10, 12]
        }

channel_position = {
    0: [650, 0],
    2: [650, 50],
    4: [650, 100],
    6: [600, 100],
    8: [600, 50],
    10: [600, 0],
    1: [0, 0],
    3: [0, 50],
    5: [0, 100],
    7: [50, 100],
    9: [50, 50],
    11: [50, 0],
    13: [150, 200], 
    15: [150, 250],
    17: [150, 300],
    19: [200, 300],
    21: [200, 250],
    23: [200, 200],
    12: [500, 200],
    14: [500, 250],
    16: [500, 300],
    18: [450, 300],
    20: [450, 250],
    22: [450, 200],
    24: [350, 400],
    26: [350, 450],
    28: [350, 500],
    25: [300, 400],
    27: [300, 450],
    29: [300, 500]
}

In [29]:
def calculate_position(row):
    probe_group = str(row['probe_group'])
    channels = channel_indices[probe_group]
    waveform = row['waveform'] 
    
    a_squared = [np.sum(waveform[:, j]**2) for j in range(len(channels))]
    
    sum_x_a = 0
    sum_y_a = 0
    sum_a = 0
    
    for j, channel in enumerate(channels):
        x_i, y_i = channel_position.get(channel, [0, 0])  
        a_i_sq = a_squared[j]
        
        sum_x_a += x_i * a_i_sq
        sum_y_a += y_i * a_i_sq
        sum_a += a_i_sq
    
    if sum_a == 0:
        return pd.Series({'position_1': 0, 'position_2': 0})
    
    x_hat = sum_x_a / sum_a
    y_hat = sum_y_a / sum_a
    return pd.Series({'position_1': x_hat, 'position_2': y_hat})

all_cluster_inf[['position_1', 'position_2']] = all_cluster_inf.apply(calculate_position, axis=1)

In [30]:
import pandas as pd
import numpy as np


current_max_neuron = 1  
all_cluster_inf['Neuron'] = None
for i in range(1, len(all_cluster_inf)):
    current_pos1 = all_cluster_inf.at[i, 'position_1']
    current_pos2 = all_cluster_inf.at[i, 'position_2']
    
    mask = (
        (all_cluster_inf.loc[:i-1, 'position_1'] - current_pos1).abs().lt(10) & 
        (all_cluster_inf.loc[:i-1, 'position_2'] - current_pos2).abs().lt(10)
    )
    
    matched = all_cluster_inf.loc[:i-1][mask]
    
    if not matched.empty:
        all_cluster_inf.at[i, 'Neuron'] = matched['Neuron'].iloc[-1]
    else:
        current_max_neuron += 1
        all_cluster_inf.at[i, 'Neuron'] = f'Neuron_{current_max_neuron}'


In [31]:
len(os.listdir("/media/ubuntu/sda/data/sort_output/mouse21/"))

11

In [32]:
neuron_date = pd.crosstab(all_cluster_inf['Neuron'], all_cluster_inf['date'])   
neuron_date[neuron_date > 1] = 1
neuron_date = neuron_date.sum(axis=1)
neuron_date = neuron_date[neuron_date == 11]
neuron_date = neuron_date.index

In [33]:
all_cluster_inf = all_cluster_inf[all_cluster_inf['Neuron'].isin(neuron_date)]
all_cluster_inf['cluster_date'] = all_cluster_inf['date']  + "_" +  all_cluster_inf['cluster']
all_spike_inf['cluster_date'] = all_spike_inf['date']  + "_" +  all_spike_inf['cluster']

all_spike_inf = all_spike_inf[all_spike_inf['cluster_date'].isin(list(all_cluster_inf['cluster_date']))]

In [34]:
import numpy as np

def calculate_position_waveform(row, channel_position, channel_indices, power=2):
    x_target = row['position_1']
    y_target = row['position_2']
    probe_group = str(row['probe_group'])
    channels = channel_indices[probe_group]  
    waveforms = row['waveform']  
    
    distances = []
    for channel in channels:
        x_channel, y_channel = channel_position.get(channel, [np.nan, np.nan])
        if np.isnan(x_channel):  
            continue
        distance = np.sqrt((x_target - x_channel)**2 + (y_target - y_channel)**2)
        distances.append(distance)
    
    if not distances:  
        return np.zeros(61)
    
    #IDW
    weights = 1 / (np.array(distances) ** power)
    if np.any(distances == 0):
        zero_idx = np.argwhere(distances == 0).flatten()
        return waveforms[:, zero_idx[0]]
    
    weights /= np.sum(weights)
    
    synthesized_waveform = np.zeros(61)
    for t in range(61): 
        weighted_sum = np.dot(waveforms[t, :], weights)
        synthesized_waveform[t] = weighted_sum
    
    return synthesized_waveform

all_cluster_inf['position_waveform'] = all_cluster_inf.apply(
    calculate_position_waveform, 
    axis=1, 
    args=(channel_position, channel_indices, 2)  
)

In [35]:
all_cluster_inf

,cluster,num_spikes,firing_rate,presence_ratio,snr,isi_violations_ratio,isi_violations_count,rp_contamination,rp_violations,sliding_rp_violation,...,drift_mad,sd_ratio,position_1,position_2,probe_group,waveform,date,Neuron,cluster_date,position_waveform
3,4,8446,3.248449,1.000000,2.252526,1.445768,119,1.000000,37,NaN,...,1.061695,2.225558,641.400962,62.899003,5,"[[-0.120943, 0.20853855, 0.1443778, 0.09690849...",062122,Neuron_3,062122_4,"[0.16436795012214697, 0.1477282516219664, 0.13..."
4,5,14721,5.661901,1.000000,6.900704,0.907831,227,1.000000,142,NaN,...,1.090121,1.415717,649.605968,54.321259,5,"[[0.018545548, -0.09148483, -0.025410857, 0.00...",062122,Neuron_3,062122_5,"[-0.08876385933132958, -0.0559571535019161, 0...."
11,14,12666,4.871520,1.000000,7.325588,1.361367,252,1.000000,134,NaN,...,0.392727,2.118095,611.278065,97.851297,5,"[[5.519553e-05, -0.028711714, -0.02628339, -0....",062122,Neuron_8,062122_14,"[-0.020219389838512663, 0.0023422957753171884,..."
33,6,13125,5.048058,1.000000,7.305398,4.145552,824,1.000000,488,NaN,...,2.748903,1.617174,642.696273,58.864307,5,"[[0.039670512, -0.3299888, -0.14168939, -0.124...",041722,Neuron_3,041722_6,"[-0.2789422852138278, -0.2708256312519234, -0...."
37,11,7141,2.746528,1.000000,6.183452,1.427628,84,1.000000,40,NaN,...,0.455770,1.343914,604.549252,89.889094,5,"[[-0.036631536, -0.18387279, -0.10663584, -0.1...",041722,Neuron_8,041722_11,"[-0.1558038269937219, -0.08481964930054964, -0..."
53,10,20996,8.075354,1.000000,7.247194,0.603558,307,0.582525,140,0.235,...,0.781837,1.488646,649.427372,52.676162,5,"[[-0.0064812163, -0.10377522, -0.027637454, 0....",072022,Neuron_3,072022_10,"[-0.10256457431903214, -0.07249148700554006, -..."
58,17,18613,7.158819,1.000000,8.584763,1.163252,465,1.000000,259,NaN,...,0.304062,2.271000,603.789528,98.614700,5,"[[-0.0025465353, -0.021629915, -0.042541184, -...",072022,Neuron_8,072022_17,"[-0.04934045238996254, -0.010755226347090275, ..."
87,9,15891,6.111900,1.000000,5.933526,0.926649,270,1.000000,166,NaN,...,1.326998,1.581079,649.329777,53.185364,5,"[[0.007814503, -0.14725718, -0.021208202, 0.03...",112022,Neuron_3,112022_9,"[-0.14460768213523292, -0.10118753169797189, -..."
91,17,15196,5.844593,0.976744,7.043097,1.422441,379,1.000000,220,NaN,...,0.989585,2.242444,604.462679,97.282262,5,"[[0.00036905776, -0.021850156, -0.06798059, -0...",112022,Neuron_8,112022_17,"[-0.05525776075018396, 0.014468249080819354, 0..."
114,5,12059,4.638059,1.000000,6.096123,0.685376,115,0.600414,47,0.255,...,0.643816,1.276441,649.287806,52.517508,5,"[[0.009046484, -0.18372701, -0.07061903, 0.006...",052222,Neuron_3,052222_5,"[-0.1817799620294813, -0.10263447976629828, -0..."


In [36]:
os.makedirs("/media/ubuntu/sda/data/filter_neuron/mouse_21/natural_image/waveform", exist_ok=True)
for neuron in all_cluster_inf['Neuron']:
    temp = all_cluster_inf[all_cluster_inf['Neuron'] == neuron]
    temp.index = temp['cluster_date']
    df_expanded = temp['position_waveform'].apply(pd.Series)
    df_expanded.to_csv(f"/media/ubuntu/sda/data/filter_neuron/mouse_21/natural_image/waveform/waveform_mean_{neuron}.csv")

In [37]:
num = 0
results = {}
folder_path = '/media/ubuntu/sda/data/filter_neuron/mouse_21/natural_image/waveform'

csv_files = [f for f in os.listdir(folder_path) if f.startswith('waveform_mean_Neuron_') and f.endswith('.csv')]

label_df = pd.DataFrame()
for csv_file in csv_files:
    df = pd.read_csv(os.path.join(folder_path, csv_file), index_col=0)
    
    from sklearn.cluster import DBSCAN
    from sklearn.decomposition import PCA
    
    pca = PCA(n_components=2)
    principal_components = pca.fit_transform(df)

    eps = 10
    min_samples = 1

    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    dbscan.fit(principal_components)

    label = pd.DataFrame(dbscan.labels_, columns=['labels'])
    label['cluster_date'] = df.index
    label['date'] = label['cluster_date'].apply(lambda x: x.split('_')[0])

    remain_label = label['labels'].value_counts()
    remain_label = remain_label[remain_label >= 11]
    for i in remain_label.index:
        temp = label[label['labels'] == i]
        if temp['date'].nunique() != 11:
            remain_label = remain_label.drop(i)
    label = label[label['labels'].isin(remain_label.index)]
    for i in label['labels'].unique():
        results[num] = label.loc[label['labels'] ==i, 'cluster_date'].values
        num += 1

In [38]:
len(results)

2

In [39]:
all_cluster_inf['Neuron'] = None
for key,item in results.items():
    all_cluster_inf.loc[all_cluster_inf['cluster_date'].isin(item), 'Neuron'] = f'Neuron_{key+1}'

In [40]:
all_cluster_inf = all_cluster_inf.dropna(subset=['Neuron'])
all_spike_inf = all_spike_inf[all_spike_inf['cluster_date'].isin(all_cluster_inf['cluster_date'].unique())]

In [41]:
all_spike_inf['Neuron'] = None
for i in range(len(all_cluster_inf)):
    all_spike_inf.loc[all_spike_inf['cluster_date'] == all_cluster_inf.iloc[i, 28], "Neuron"] = all_cluster_inf.iloc[i, 27]

In [42]:
all_cluster_inf['neuron_date'] = all_cluster_inf['date'] + "_" + all_cluster_inf['Neuron']
all_spike_inf['neuron_date'] = all_spike_inf['date'] + "_" + all_spike_inf['Neuron']

In [44]:
waveform_mean = pd.DataFrame()
csv_files = [f for f in os.listdir('/media/ubuntu/sda/data/filter_neuron/mouse_21/natural_image/waveform') if f.startswith('waveform_mean_Neuron_') and f.endswith('.csv')]
for csv_file in csv_files:
    df = pd.read_csv(os.path.join('/media/ubuntu/sda/data/filter_neuron/mouse_21/natural_image/waveform', csv_file), index_col=0)
    waveform_mean = pd.concat((waveform_mean, df), axis=0)

waveform_mean = waveform_mean.loc[list(all_cluster_inf['cluster_date'])]

In [45]:
all_cluster_inf = all_cluster_inf.set_index('cluster_date')
all_cluster_inf = all_cluster_inf.join(waveform_mean, how="right")

In [49]:
temp

,cluster,num_spikes,firing_rate,presence_ratio,snr,isi_violations_ratio,isi_violations_count,rp_contamination,rp_violations,sliding_rp_violation,...,51,52,53,54,55,56,57,58,59,60
cluster_date,,,,,,,,,,,,,,,,,,,,,
062122_5,5,14721,5.661901,1.0,6.900704,0.907831,227,1.000000,142,NaN,...,0.307001,0.281617,0.227595,0.147121,0.093059,0.075178,0.069957,0.035060,-0.003214,-0.039600
041722_6,6,13125,5.048058,1.0,7.305398,4.145552,824,1.000000,488,NaN,...,0.080454,0.105709,0.198383,0.253383,0.263862,0.287832,0.312823,0.238381,0.079470,-0.023237
072022_10,10,20996,8.075354,1.0,7.247194,0.603558,307,0.582525,140,0.235,...,0.303608,0.253769,0.209907,0.168779,0.121611,0.062009,0.014000,-0.020648,-0.042611,-0.059145
112022_9,9,15891,6.111900,1.0,5.933526,0.926649,270,1.000000,166,NaN,...,0.269238,0.284099,0.296489,0.260550,0.186901,0.114533,0.052597,-0.001058,-0.047417,-0.062530
052222_5,5,12059,4.638059,1.0,6.096123,0.685376,115,0.600414,47,0.255,...,0.212227,0.223569,0.218067,0.171523,0.117807,0.067660,-0.001944,-0.074598,-0.085425,-0.066749
082222_12,12,22963,9.211768,1.0,7.554116,0.865126,549,1.000000,292,NaN,...,0.327561,0.274664,0.214316,0.146561,0.068514,0.000367,-0.040661,-0.054179,-0.052441,-0.043904
011623_4,4,13244,5.093827,1.0,6.490518,1.067257,216,1.000000,97,NaN,...,0.308584,0.285634,0.256560,0.231185,0.182454,0.117991,0.060984,-0.002667,-0.060885,-0.086727
092422_10,10,5930,2.280760,1.0,6.614552,3.327194,135,1.000000,157,NaN,...,0.245187,0.240688,0.209289,0.157665,0.099897,0.071201,0.021898,-0.056099,-0.107242,-0.083691
102022_4,4,15005,6.079346,1.0,6.768022,0.869687,238,1.000000,129,NaN,...,0.287792,0.240711,0.210395,0.178252,0.136476,0.107548,0.057569,-0.013951,-0.064291,-0.063301


In [52]:
from scipy.stats import pearsonr
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import colorsys
from matplotlib.backends.backend_pdf import PdfPages

def generate_base_colors(n, saturation=0.6, lightness=0.5):
    hues = np.linspace(0, 1, n, endpoint=False)  
    base_colors = []
    for h in hues:
        rgb = colorsys.hls_to_rgb(h, lightness, saturation)
        base_colors.append(rgb)
    return base_colors

n_neurons = len(all_cluster_inf['Neuron'].unique())
base_palette = generate_base_colors(n_neurons) 


def get_gradient_palette(base_color, n_levels=16, reverse=False):
    palette = sns.light_palette(base_color, n_levels, reverse=reverse)
    return palette


with PdfPages('figure/cluster_view.pdf') as pdf:
    n_neurons = len(all_cluster_inf['Neuron'].unique())
    base_palette = generate_base_colors(n_neurons)
    
    for idx, neuron in enumerate(all_cluster_inf['Neuron'].unique()):
        temp = all_cluster_inf[all_cluster_inf['Neuron'] == neuron]
        
        current_base_color = base_palette[idx]
        line_palette = get_gradient_palette(current_base_color, n_levels=25, reverse=False)
        
        fig, ax = plt.subplots(figsize=(1.5, 1.5))
        for i in range(11):
            sns.lineplot(
                x=range(50),
                y=temp.iloc[i, 41:],
                color=line_palette[i + 6],  
                ax=ax
            )
        
        ax.set_ylabel("Amplitude")
        ax.set_xticks([])
        ax.set_title(neuron)
        pdf.savefig(fig)
        plt.close(fig)

In [53]:
all_cluster_inf.to_csv('/media/ubuntu/sda/data/filter_neuron/mouse_21/natural_image/cluster_inf.tsv', sep = '\t')
all_spike_inf.to_csv("/media/ubuntu/sda/data/filter_neuron/mouse_21/natural_image/spike_inf.tsv", sep='\t')

In [54]:
all_cluster_inf = pd.read_csv("cluster_inf.tsv", sep = '\t')